In [66]:
import hashlib

string = "CitricSheep"
ascii_list = [ord(char) for char in string]
multiplied_list = [value * len(string) for value in ascii_list]
sum_of_values = sum(multiplied_list)

hash_object = hashlib.sha256(str(sum_of_values).encode())
hex_hash = hash_object.hexdigest()

print(hex_hash)

44aad44806272b73f17ba4027a0645985b0851ea5723d4d20c46ae14d6be68b4


In [24]:
## Run if it´s necesary to install
# !pip install pyarrow

## Import packages

In [25]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [26]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [27]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/Ivan/MLOps_ZoomCamp/2-Experiment-tracking/notebooks/mlruns/1', creation_time=1687303853945, experiment_id='1', last_update_time=1687303853945, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## Develop a funtion to load the data

In [28]:
def read_dataframe(filename):
    
    df = pd.read_parquet(filename)

    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df["duration"].apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration >=1) & (df.duration <=60)]
    categorical = ["PULocationID","DOLocationID"]
    df[categorical] = df[categorical].astype(str)
    
    return df

### loading the data

In [29]:
# MLOps_ZoomCamp/data/yellow_tripdata_2022-01.parquet
df_train = read_dataframe ("../data/yellow_tripdata_2022-01.parquet")
df_val = read_dataframe("../data/yellow_tripdata_2022-02.parquet")

In [30]:
len(df_train), len(df_val)

(2421440, 2918187)

In [31]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

## Split the data into trainning and validation data

In [32]:
categorical = ["PU_DO"] #["PULocationID","DOLocationID"]
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient = "records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient = "records")
X_val = dv.transform(val_dicts)

### Adjust target Variable


In [33]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

## Fitting some models

### Fit a linear model

In [34]:
lr = LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val,y_pred, squared=False)

5.414844188813796

### Save the best model

In [35]:
with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

### See the actual Y and predicted Y

In [36]:
# sns.distplot(y_pred, label = "prediction")
# sns.distplot(y_train, label = "actual")

# plt.legend();

### Fit a lasso model

In [37]:
# la = Lasso(alpha=0.01)
# la.fit(X_train,y_train)

# y_pred = la.predict(X_val)

# mean_squared_error(y_val,y_pred, squared=False)

### Fit a ridge model - tracking with mlflow

In [38]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Ivan")
    
    mlflow.log_param("train-data-path","../data/yellow_tripdata_2022-01.parquet")
    mlflow.log_param("valid-data-path","../data/yellow_tripdata_2022-02.parquet")
    
    
    
    alpha = 0.1
    
    mlflow.log_param("alpha", alpha)
    
    
    
    lg = Ridge(alpha)
    lg.fit(X_train,y_train)

    y_pred = lg.predict(X_val)

    rmse = mean_squared_error(y_val,y_pred, squared=False)
    
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path = "models/lin_reg.bin", artifact_path="models_pickle")

# improving the model applying Xgboost Model

## Hyperparameter Optimizaiton Tracking:

In [39]:
%%time
import xgboost as xgb

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

CPU times: user 271 ms, sys: 39.5 ms, total: 310 ms
Wall time: 530 ms


In [40]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [26]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}



In [27]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [ ]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

## Autologging:

Instead of logging the parameters by "Hand" by specifiying the logged parameters and passing them. We may use the Autologging feature in MLflow. There are two ways to use Autologging; First by enabling it globally in the code/Notebook using

In [18]:
mlflow.autolog()

2023/06/21 00:47:18 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2023/06/21 00:47:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2023/06/21 00:47:19 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2023/06/21 00:47:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


or by enabling the framework-specific autologger; ex with XGBoost:

In [ ]:
params = {
    "learning_rate" : 0.09726255724350065,
    "max_depth": 51,
    "min_child_weight": 19.788154446519542,
    "objective": "reg:linear",
    "reg_alpha": 0.0802169697770356,
    "reg_lambda": 0.003109431432408728,
    "seed": 42
}

# mlflow.xgboost.autolog()

booster = xgb.train(
params=params,
dtrain=train,
num_boost_round=10,
evals=[(valid, 'validation')],
early_stopping_rounds=50)

2023/06/21 00:47:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '84c726ee884342eab3fff8605afffbd7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


Both must be done before running the experiments.

The autologger then not only stores the model parameters for ease of use, it also stores other files inside the model (can be specified) folder inside our experiment artifact folder, these files include:

conda.yaml and requirements.txt: Files which define the current envrionment for use with either conda or pip respectively MLmodel an internal MLflow file for organization Other framework-specific files such as the model itself

## Saving Models:


In [44]:
with mlflow.start_run():
    
    best_params = {
    "learning_rate" : 0.09726255724350065,
    "max_depth": 51,
    "min_child_weight": 19.788154446519542,
    "objective": "reg:linear",
    "reg_alpha": 0.0802169697770356,
    "reg_lambda": 0.003109431432408728,
    "seed": 42
    }
    
    mlflow.log_params(best_params)

    # mlflow.xgboost.autolog()

    booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=50,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50)
    
    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val,y_pred, squared=False)
    mlflow.log_metric("rmse",rmse)
    |
    with open("models/preprocessor.b","wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[02:44:35] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:15.09843
[1]	validation-rmse:13.91454
[2]	validation-rmse:12.85768
[3]	validation-rmse:11.91633
[4]	validation-rmse:11.07898
[5]	validation-rmse:10.33584
[6]	validation-rmse:9.67806
[7]	validation-rmse:9.09688
[8]	validation-rmse:8.58475
[9]	validation-rmse:8.13432
[10]	validation-rmse:7.73941
[11]	validation-rmse:7.39356
[12]	validation-rmse:7.09121
[13]	validation-rmse:6.82782
[14]	validation-rmse:6.59851
[15]	validation-rmse:6.39880
[16]	validation-rmse:6.22547
[17]	validation-rmse:6.07457
[18]	validation-rmse:5.94358
[19]	validation-rmse:5.83003
[20]	validation-rmse:5.73120
[21]	validation-rmse:5.64508
[22]	validation-rmse:5.57005
[23]	validation-rmse:5.50462
[24]	validation-rmse:5.44759
[25]	validation-rmse:5.39785
[26]	validation-rmse:5.35398
[27]	validation-rmse:5.31569
[28]	validation-rmse:5.28197
[29]	validation-rmse:5.25234
[30]	validation

## Loading Models:


In [45]:
logged_model = 'runs:/2958350c12f843c8973ee4db46c753df/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

2023/06/21 02:57:32 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[02:57:32] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [46]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 2958350c12f843c8973ee4db46c753df

We can use the model to make predictions with multiple ways depending on what we need:

 - We may load the model as a Spark UDF (User Defined Function) for use with Spark Dataframes
 - We may load the model as a MLflow PyFuncModel structure, to then use to predict data in a Pandas DataFrame, NumPy Array or SciPy Sparse Array. The obtained interface is general for all models from all frameworks
 - We may load the model as is, i.e: load the XGBoost model as an XGBoost model and treat it as such

The first two methods are explained briefly in the MLflow artifacts page for each run, for the latter we may use (XGBoost example):

In [48]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

[02:58:47] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


the resultant xgboost_model is an XGBoost Booster object which behaves like any XGBoost model. We can predict as normal and even use XGBoost Booster functions such as get_fscore for feature importance.

In [49]:
y_pred = xgboost_model.predict(valid)

In [50]:
y_pred[:10]

array([14.791502 , 20.216396 , 28.003994 , 26.085732 , 31.809734 ,
       13.911249 , 24.460573 ,  5.2154083, 18.154009 , 16.186596 ],
      dtype=float32)

## Saving Artifacts with the Model:

Sometimes we may want to save some artifacts with the model, for example in our case we may want to save the DictVectorizer object with the model for inference (subsequently testing as well). In that case we save the artifact as:

In [ ]:
mlflow.log_artifact("vectorizer.pkl", artifact_path="extra_artifacts")

Where vectorizer.pkl is the vectorizer stored in a Pickle file and extra_artifacts the folder within the artifacts of the model where the file is stored.

## Interacting with MLflow through the Tracking Client:

In order to automate the process of registering/promoting/demoting models, we use the Tracking Client API initialized as described above:

In [53]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

### Selecting runs:

We can search for runs by ascending order of metric score using the API by:



In [55]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',    # Experiment ID we want
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

We can then get information about the selected runs from the resulting runs enumerator:



In [57]:
for run in runs:
    print(f"rund id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

rund id: 2958350c12f843c8973ee4db46c753df, rmse: 5.0268
rund id: 4da5f7c8f38447f6a2300db84cd7f98d, rmse: 5.0268
rund id: 23bdd3c6e5d9411fb6f5d241019ec7dc, rmse: 6.0074
rund id: 338bf18eaaa4447281bb5ad0b82a65e2, rmse: 6.0075
rund id: acfa6db581684619bd9bb2e85bb903f8, rmse: 6.0075


## Interacting with the Model Registry:

We can add a run model to a registry using:



In [58]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "338bf18eaaa4447281bb5ad0b82a65e2"
model_uri = f"runs:/{run_id}/models"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

2023/06/22 00:45:57 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/06/22 00:45:57 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/06/22 00:45:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1687394757541, current_stage='None', description=None, last_updated_timestamp=1687394757541, name='nyc-taxi-regressor', run_id='338bf18eaaa4447281bb5ad0b82a65e2', run_link=None, source='/home/Ivan/MLOps_ZoomCamp/2-Experiment-tracking/notebooks/mlruns/1/338bf18eaaa4447281bb5ad0b82a65e2/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=3>

we can get the models in a model registry:



In [60]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


promote a model to staging:


In [61]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1687394757541, current_stage='Staging', description=None, last_updated_timestamp=1687394986431, name='nyc-taxi-regressor', run_id='338bf18eaaa4447281bb5ad0b82a65e2', run_link=None, source='/home/Ivan/MLOps_ZoomCamp/2-Experiment-tracking/notebooks/mlruns/1/338bf18eaaa4447281bb5ad0b82a65e2/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=3>

update the description of a model:



In [63]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1687394757541, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-06-22', last_updated_timestamp=1687395121552, name='nyc-taxi-regressor', run_id='338bf18eaaa4447281bb5ad0b82a65e2', run_link=None, source='/home/Ivan/MLOps_ZoomCamp/2-Experiment-tracking/notebooks/mlruns/1/338bf18eaaa4447281bb5ad0b82a65e2/artifacts/models', status='READY', status_message=None, tags={}, user_id=None, version=3>